### Partes de un tensor

La ventaja que tiene sobre NumPy es que es acelerado por GPU y **autograd**.

Los tensores de imágenes se leen así: (C, H, W)

C: Número de canales.

H: Altura de la imagen.

W: Anchura de la imagen.

In [27]:
import torch
from torchvision import datasets 
from torchvision.transforms import ToTensor # Convierte una imagen a tensor
import matplotlib.pyplot as plt


In [28]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.9.0+cu126
True
Tesla T4


In [29]:
# training_data y test_data son objetos de Dataset

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor() # Se aplica a cada imagen cuando se accede, no a todas de una
)

In [30]:
training_data[0][0].size()

torch.Size([1, 28, 28])

In [31]:
import numpy as np
from collections import Counter # Estructura de datos para contar frecuencias de elementos

labels = [training_data[i][1] for i in range(len(training_data))]
counts = Counter(labels)

print(counts)

Counter({1: 6742, 7: 6265, 3: 6131, 2: 5958, 9: 5949, 0: 5923, 6: 5918, 8: 5851, 4: 5842, 5: 5421})


DataLoader: es un iterador inteligente de PyTorch. Toma un Dataset y lo convierte en lotes (batches) 

iter(train_loader): convierte el DataLoader en un iterador. Es como decir for batch in train_loader

In [32]:
# DATALOADER 
from torch.utils.data import DataLoader

batch_size = 64 # Cuántos ejemplos se procesan juntos en una sola pasada por el modelo
                # El modelo "ve" 64 imágenes al mismo tiempo
                

train_loader = DataLoader(
    training_data,
    batch_size=batch_size,
    shuffle=True # Mezcla los datos en cada época
)

test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False # Para que los resultados sean reproducibles
)

X, y = next(iter(train_loader)) 
print(X.shape) # (64, 1, 28, 28) 
print(y.shape) # (64,)


torch.Size([64, 1, 28, 28])
torch.Size([64])


### Capas

#### Capa de escaneo

nn.Conv2d: Define capas de escaneo para imágenes. Pieza fundamental de las CNN. Su trabajo es detectar patrones (bordes, texturas o formas) deslizando un kernel sobre la imagen. Si detecta algo envía una señal fuerte. El resultado es una nueva "imagen" (mapa de características) donde se resaltan los patrones encontrados.

Primeros dos argumentos: input channels (cuántos canales tiene la imagen de entrada) y output channels/filters (cuántos filtros se aplicarán, por ende, cuántos mapas de características se obtendrán).

#### Capa de reducción

Hace la imagen más pequeña para que la red sea más rápida y se enfoque solo en lo más importante. Max pooling es una técnica que divide la imagen en una cuadrícula de pequeños cuadros y mira cada cuadro y se queda solo con el número más grande, descartando los demás. Se reduce el tamaño y evita el ruido.

Parámetros: tamaño cuadrado del kernel y cuánto se "salta" la ventana cada vez que se mueve.

#### Capas densas o lineales

Capa donde todas las neuronas de la entrada se conectan con todas las neuronas de la salida. A diferencia de las convolucionales, estas miran toda la información junta para clasificarla. 

Parámetros: entrada y salida (10 neuronas para 10 dígitos). 

### Otros aspectos

x.view(x.size(0), -1) 

x.view() es la función que cambia la forma (dimensión) de un tensor sin mover sus datos en la memoria. Como reshape en NumPy

- x.size(0) mantiene el tamaño del batch

- -1 estira todos los píxeles de los 32 mapas de 7x7 en una sola fila larga.


In [33]:
import torch.nn as nn # Define piezas estructurales de la red que tienen parámetros entrenables (pesos y sesgos)
import torch.nn.functional as F # Aplica funciones sobre los datos que no tienen parámetros propios. Son acciones matemáticas que no necesitan recordad nada de la iteración anterior

class SimpleCNN(nn.Module): # nn.Module es la madre de todas las redes
    def __init__(self):
        super().__init__() # Conecta la clase con todas las funciones internas de PyTorch
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    # Define el camino que siguen los datos desde que entran hasta que se obtiene un resultado
    def forward(self, x):
        
        # Capas de extracción (Convolución + Activación + Pooling)
        x = self.pool(F.relu(self.conv1(x)))  # (N,16,14,14)
        x = self.pool(F.relu(self.conv2(x)))  # (N,32,7,7)
        
        # Aplanado o flatten
        x = x.view(x.size(0), -1)   
        
        # Capas de decisión (fully connected)           
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
model = SimpleCNN().to(device)

# Componentes que hacen que la red aprenda
criterion = nn.CrossEntropyLoss() # Se define la función de pérdida y mide qué tan mal lo está haciendo la red
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # Define la optimización, encargado de ajustar los pesos para no cometer el mismo error


cuda


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for X, y in loader:
        X = X.to(device) # Mueve los datos a la GPU
        y = y.to(device)

        # BACKPROPAGATION
        optimizer.zero_grad() # Borra los gradientes
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward() # Viaja hasta la primera capa y calcula cuánta responsabilidad tuvo cada peso en el error final
        optimizer.step() # Cambia ligeramente los pesos de las neuronas para minimizar el error

        total_loss += loss.item()

    return total_loss / len(loader)


In [ ]:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad(): # with es un manejador de contexto. Específicamente aquí dice que durante este bloque que aplique esta configuración especial (no calcular gradientes)
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            outputs = model(X)
            _, predicted = torch.max(outputs, 1)

            total += y.size(0)
            correct += (predicted == y).sum().item()

    return correct / total


In [40]:
for epoch in range(3):
    loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    acc = evaluate(model, test_loader, device)

    print(f"Epoch {epoch+1}: loss={loss:.4f}, acc={acc:.4f}")


Epoch 1: loss=0.2299, acc=0.9756
Epoch 2: loss=0.0645, acc=0.9845
Epoch 3: loss=0.0435, acc=0.9845


In [38]:
class Robot:
    def __init__(self, nombre):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola, soy {self.nombre}")
        
    def dinero(self, din):
        self.din = din
        
    def cant(self):
        print(self.din)
        
rob = Robot("ola")
rob.nombre


'ola'

In [52]:

# 1. Convertimos a tensores
predicciones = torch.tensor([1, 2, 1, 1, 40])
etiquetas    = torch.tensor([1, 1, 1, 1, 40])

# 2. Comparamos (esto crea [True, False, True, True, False])
comparacion = (predicciones == etiquetas)

# 3. Sumamos los True
aciertos = comparacion.sum()

print(aciertos.item())

print(len(y)) 


4
64
